In [1]:
# Experiment: 2021/03/14
# Usage: Dp analysis of mnist neural network.
# Detail: Check the notion notes in HE-DP project.

from dpsgd_keras_slow import *

dpsgd = False # add dp noise or not 
learning_rate = 0.05
noise_multiplier = 8
l2_norm_clip = 3
batch_size = 1024
epochs = 500
microbatch_size = 16
num_parameters = 0
privacy_budget = []
delta = 1e-5  # it is recommended to use delta~=1/dataset_size
model_dir = None

In [2]:
import tensorflow as tf
tf.__version__

'2.6.0'

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7,8'

In [4]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']


In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [7]:
train, test = tf.keras.datasets.cifar10.load_data()

In [8]:
def load_cifar10():
  """Loads MNIST and preprocesses to combine training and validation data."""
  train, test = tf.keras.datasets.cifar10.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  train_data = np.array(train_data, dtype=np.float32) / 255
  test_data = np.array(test_data, dtype=np.float32) / 255

  train_data = train_data.reshape((train_data.shape[0], 32, 32, 3))
  test_data = test_data.reshape((test_data.shape[0], 32, 32, 3))

  train_labels = np.array(train_labels, dtype=np.int32)
  test_labels = np.array(test_labels, dtype=np.int32)

  train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
  test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

  assert train_data.min() == 0.
  assert train_data.max() == 1.
  assert test_data.min() == 0.
  assert test_data.max() == 1.

  return train_data, train_labels, test_data, test_labels

In [9]:
# Perturnbing the input dataset, and collect the accuracy-step
class GaussianNoiseLayer(tf.keras.layers.Layer):
    def __init__(self, l2_norm_clip, noise_multiplier, *args, **kwargs):
        super(GaussianNoiseLayer, self).__init__(*args, **kwargs)
        self._l2_norm_clip = l2_norm_clip
        self._noise_multiplier = noise_multiplier

    def build(self, input_shape):
        pass
    
    def call(self, x):
        # Clip gradients to given l2_norm_clip.
        def clip_features(x):
            return tf.clip_by_global_norm([x], self._l2_norm_clip)[0][0]

        clipped_features = tf.map_fn(clip_features, x)
        
        # Add noise to summed gradients.
        noise_stddev = self._l2_norm_clip * self._noise_multiplier
        noise = tf.random.normal(tf.shape(input=clipped_features), stddev=noise_stddev)
        return clipped_features + noise

In [10]:
#tf.clip_by_global_norm([tf.Variable([[2,3],[4,5]],shape=[2,2],dtype=tf.float32)], 4.4)[0][0]

In [11]:
# Models 



def build_models(noise_layer_name):
    if noise_layer_name =="cifar10+untied_bias+noise_input":
        model = tf.keras.Sequential([
            GaussianNoiseLayer(l2_norm_clip, noise_multiplier, input_shape=(32, 32, 3)),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False),
            BiasLayer()
        ])
    elif noise_layer_name == "cifar10+tied_bias+noise_input":
        model = tf.keras.Sequential([
            GaussianNoiseLayer(l2_norm_clip, noise_multiplier, input_shape=(32, 32, 3)),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False),
            BiasLayer()
        ])
    elif noise_layer_name == "cifar10+untied_bias":
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False),
            BiasLayer()
        ])
#     elif noise_layer_name == "cifar10+tied_bias":
#             model = tf.keras.Sequential([
#             tf.keras.layers.Conv2D(16, 5,
#                                  strides=1,
#                                  padding='valid',
#                                  activation=None,
#                                  input_shape=(32, 32, 3), use_bias=False),
#             TiedBiasLayer(),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.MaxPool2D((2,2), 2),
#             tf.keras.layers.Conv2D(16, 5,
#                                  strides=1,
#                                  padding='valid',
#                                  activation=None, use_bias=False),
#             TiedBiasLayer(),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.MaxPool2D((2,2), 2),
#             tf.keras.layers.Flatten(),
#             tf.keras.layers.Dense(100, activation=None, use_bias=False),
#             BiasLayer(),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Dense(10, activation=None, use_bias=False),
#             BiasLayer()
#         ])
    elif noise_layer_name == "cifar10+tied_bias":
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False),
            BiasLayer()
        ])
    elif noise_layer_name == "cifar10":
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False),
            BiasLayer()
        ])  
    elif noise_layer_name == "sphinx+cifar10+untied_bias":
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False, name="Considered1"),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered2"),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered3"),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered4"),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered5"),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered6"),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered7"),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False, name="Considered8"),
            BiasLayer()
        ])
        
        
    elif noise_layer_name == "sphinx+cifar10+tied_bias":
        model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, 5,
                             strides=1,
                             padding='valid',
                             activation=None,
                             input_shape=(32, 32, 3), use_bias=False, name="Considered1"),
        TiedBiasLayer(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPool2D((2,2), 2),
        tf.keras.layers.Conv2D(16, 5,
                             strides=1,
                             padding='valid',
                             activation=None, use_bias=False, name="Considered2"),
        TiedBiasLayer(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPool2D((2,2), 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(100, activation=None, use_bias=False, name="Considered3"),
        BiasLayer(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Dense(10, activation=None, use_bias=False, name="Considered4"),
        BiasLayer()
    ])
#     elif noise_layer_name == "sphinx+cifar10+tied_bias":
#         model = tf.keras.Sequential([
#             tf.keras.layers.Conv2D(64, 3,
#                                  strides=1,
#                                  padding='same',
#                                  activation=None,
#                                  input_shape=(32, 32, 3), use_bias=False, name="Considered1"),
#             TiedBiasLayer(),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Conv2D(64, 3,
#                                  strides=1,
#                                  padding='same',
#                                  activation=None, use_bias=False, name="Considered2"),
#             TiedBiasLayer(),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.MaxPool2D((2,2), 2),
#             tf.keras.layers.Conv2D(64, 3,
#                                  strides=1,
#                                  padding='same',
#                                  activation=None, use_bias=False, name="Considered3"),
#             TiedBiasLayer(),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Conv2D(64, 3,
#                                  strides=1,
#                                  padding='same',
#                                  activation=None, use_bias=False, name="Considered4"),
#             TiedBiasLayer(),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.MaxPool2D((2,2), 2),
#             tf.keras.layers.Conv2D(64, 3,
#                                  strides=1,
#                                  padding='same',
#                                  activation=None, use_bias=False, name="Considered5"),
#             TiedBiasLayer(),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Conv2D(64, 1,
#                                  strides=1,
#                                  padding='same',
#                                  activation=None, use_bias=False, name="Considered6"),
#             TiedBiasLayer(),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Conv2D(16, 1,
#                                  strides=1,
#                                  padding='same',
#                                  activation=None, use_bias=False, name="Considered7"),
#             TiedBiasLayer(),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Flatten(),
#             tf.keras.layers.Dense(10, activation=None, use_bias=False, name="Considered8"),
#             BiasLayer()
#         ])
    elif noise_layer_name == "ALLnoise+cifar10+untied_bias":
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False, name="Considered1"),
            BiasLayer(name="Considered9"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered2"),
            BiasLayer(name="Considered10"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered3"),
            BiasLayer(name="Considered11"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered4"),
            BiasLayer(name="Considered12"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered5"),
            BiasLayer(name="Considered13"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered6"),
            BiasLayer(name="Considered14"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered7"),
            BiasLayer(name="Considered15"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False, name="Considered8"),
            BiasLayer(name="Considered16")
        ])
    elif noise_layer_name == "ALLnoise+cifar10+tied_bias":
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(16, 5,
                                 strides=1,
                                 padding='valid',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False, name="Considered1"),
            TiedBiasLayer(name="Considered7"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(16, 5,
                                 strides=1,
                                 padding='valid',
                                 activation=None, use_bias=False, name="Considered2"),
            TiedBiasLayer(name="Considered8"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(100, activation=None, use_bias=False, name="Considered3"),
            BiasLayer(name="Considered4"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False, name="Considered5"),
            BiasLayer(name="Considered6")
         ])
#     elif noise_layer_name == "ALLnoise+cifar10+tied_bias":
#         model = tf.keras.Sequential([
#             tf.keras.layers.Conv2D(64, 3,
#                                  strides=1,
#                                  padding='same',
#                                  activation=None,
#                                  input_shape=(32, 32, 3), use_bias=False, name="Considered1"),
#             TiedBiasLayer(name="Considered9"),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Conv2D(64, 3,
#                                  strides=1,
#                                  padding='same',
#                                  activation=None, use_bias=False, name="Considered2"),
#             TiedBiasLayer(name="Considered10"),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.MaxPool2D((2,2), 2),
#             tf.keras.layers.Conv2D(64, 3,
#                                  strides=1,
#                                  padding='same',
#                                  activation=None, use_bias=False, name="Considered3"),
#             TiedBiasLayer(name="Considered11"),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Conv2D(64, 3,
#                                  strides=1,
#                                  padding='same',
#                                  activation=None, use_bias=False, name="Considered4"),
#             TiedBiasLayer(name="Considered12"),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.MaxPool2D((2,2), 2),
#             tf.keras.layers.Conv2D(64, 3,
#                                  strides=1,
#                                  padding='same',
#                                  activation=None, use_bias=False, name="Considered5"),
#             TiedBiasLayer(name="Considered13"),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Conv2D(64, 1,
#                                  strides=1,
#                                  padding='same',
#                                  activation=None, use_bias=False, name="Considered6"),
#             TiedBiasLayer(name="Considered14"),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Conv2D(16, 1,
#                                  strides=1,
#                                  padding='same',
#                                  activation=None, use_bias=False, name="Considered7"),
#             TiedBiasLayer(name="Considered15"),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Flatten(),
#             tf.keras.layers.Dense(10, activation=None, use_bias=False, name="Considered8"),
#             BiasLayer(name="Considered16")
#         ])
    else:
        model = None
    return model

In [12]:
build_models("cifar10+untied_bias+noise_input").summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise_layer (Gaussi (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1728      
_________________________________________________________________
bias_layer (BiasLayer)       (None, 32, 32, 64)        65536     
_________________________________________________________________
re_lu (ReLU)                 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36864     
_________________________________________________________________
bias_layer_1 (BiasLayer)     (None, 32, 32, 64)        65536     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32, 32, 64)        0

In [13]:
build_models("cifar10+untied_bias").summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 64)        1728      
_________________________________________________________________
bias_layer_8 (BiasLayer)     (None, 32, 32, 64)        65536     
_________________________________________________________________
re_lu_7 (ReLU)               (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 64)        36864     
_________________________________________________________________
bias_layer_9 (BiasLayer)     (None, 32, 32, 64)        65536     
_________________________________________________________________
re_lu_8 (ReLU)               (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)       

In [14]:
build_models("cifar10+tied_bias").summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 32, 32, 64)        1728      
_________________________________________________________________
tied_bias_layer (TiedBiasLay (None, 32, 32, 64)        64        
_________________________________________________________________
re_lu_14 (ReLU)              (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 32, 32, 64)        36864     
_________________________________________________________________
tied_bias_layer_1 (TiedBiasL (None, 32, 32, 64)        64        
_________________________________________________________________
re_lu_15 (ReLU)              (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 64)       

In [15]:
build_models("cifar10").summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 32, 32, 64)        1728      
_________________________________________________________________
re_lu_21 (ReLU)              (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 32, 32, 64)        36864     
_________________________________________________________________
re_lu_22 (ReLU)              (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 16, 16, 64)        36864     
_________________________________________________________________
re_lu_23 (ReLU)              (None, 16, 16, 64)       

In [16]:
#def main_simple(unused_argv):
logging.set_verbosity(logging.INFO)
if dpsgd and batch_size % microbatch_size != 0:
    raise ValueError('Number of microbatches should divide evenly batch_size')

# Load training and test data.
train_data, train_labels, test_data, test_labels = load_cifar10()

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=(epochs * train_data.shape[0]) // microbatch_size // 1000,
    decay_rate=0.998)

In [ ]:
# check the training accuracy
origin_dpsgd = dpsgd
parameter_list = {}
accuracies = {}
#for model_type in ["mnist", "mnist+tied_bias", "mnist+untied_bias"]:
for model_type in ["cifar10+tied_bias"]:
    file_name = model_type + "_accuracy"
    if model_type == "cifar10+tied_bias+noise_input":
        dpsgd = False
    else:
        dpsgd = origin_dpsgd
    model = build_models(model_type)
    if dpsgd:
        optimizer = FixedDPKerasSGDOptimizer(
            batch_size = batch_size,
            num_parameters = num_parameters,
            l2_norm_clip=l2_norm_clip,
            noise_multiplier=noise_multiplier,
            var_list = model.trainable_variables,
            #num_microbatches=batch_size//microbatch_size,
            microbatch_size = microbatch_size,
            learning_rate=learning_rate)
        # Compute vector of per-example loss rather than its mean over a minibatch.
        loss = tf.keras.losses.CategoricalCrossentropy(
            from_logits=True, reduction=tf.losses.Reduction.NONE)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
        loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # Compile model with Keras
    checkpoint_filepath = "./dp_data/"+file_name
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      save_weights_only=True,
      monitor='val_accuracy',
      mode='max',
      save_best_only=True)
    #early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    ##    monitor='val_loss', min_delta=0, patience=5, verbose=0,
    #    mode='auto', baseline=None, restore_best_weights=True
    #)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    # Train model with Keras
    if dpsgd:
        history = model.fit(train_data, train_labels,
                epochs=epochs,
                validation_data=(test_data, test_labels),
                batch_size=microbatch_size, 
                callbacks = [model_checkpoint_callback], workers=1) # , early_stopping_callback
    else:
        history = model.fit(train_data, train_labels,
                epochs=epochs,
                validation_data=(test_data, test_labels),
                batch_size=batch_size, 
                callbacks = [model_checkpoint_callback], workers=1) # , early_stopping_callback
    
    #evaluated_result = model.evaluate(
    #    x=test_data, y=test_labels, batch_size=None, verbose=1, sample_weight=None, steps=None,
    #    callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False,
    #    return_dict=True)
    if model_type not in accuracies.keys():
        accuracies[model_type] = []
    accuracies[model_type].append(history)
    #accuracies[model_type].append(evaluated_result["accuracy"])

# Compute the privacy budget expended.
#if dpsgd:
#    eps = compute_epsilon(epochs * 60000 // batch_size)
#    print('For delta=1e-5, the current epsilon is: %.2f' % eps)
#else:
#    print('Trained with vanilla non-private SGD optimizer')

Epoch 1/500
49/49 [==============================] - 8s 67ms/step - loss: 2.3020 - accuracy: 0.1198 - val_loss: 2.2996 - val_accuracy: 0.1157
Epoch 2/500
49/49 [==============================] - 2s 44ms/step - loss: 2.2958 - accuracy: 0.1547 - val_loss: 2.2903 - val_accuracy: 0.1421
Epoch 3/500
49/49 [==============================] - 2s 46ms/step - loss: 2.2740 - accuracy: 0.1875 - val_loss: 2.2435 - val_accuracy: 0.1709
Epoch 4/500
49/49 [==============================] - 2s 43ms/step - loss: 2.2098 - accuracy: 0.2013 - val_loss: 2.1346 - val_accuracy: 0.2502
Epoch 5/500
49/49 [==============================] - 2s 45ms/step - loss: 2.1028 - accuracy: 0.2499 - val_loss: 2.0040 - val_accuracy: 0.2924
Epoch 6/500
49/49 [==============================] - 2s 44ms/step - loss: 2.0428 - accuracy: 0.2702 - val_loss: 2.4428 - val_accuracy: 0.1673
Epoch 7/500
49/49 [==============================] - 2s 43ms/step - loss: 2.0237 - accuracy: 0.2774 - val_loss: 1.9270 - val_accuracy: 0.3080
Epoch 

49/49 [==============================] - 2s 44ms/step - loss: 0.9455 - accuracy: 0.6688 - val_loss: 1.0218 - val_accuracy: 0.6428
Epoch 59/500
49/49 [==============================] - 2s 44ms/step - loss: 0.9356 - accuracy: 0.6710 - val_loss: 0.9931 - val_accuracy: 0.6503
Epoch 60/500
49/49 [==============================] - 2s 44ms/step - loss: 0.9287 - accuracy: 0.6719 - val_loss: 0.9975 - val_accuracy: 0.6526
Epoch 61/500
49/49 [==============================] - 2s 43ms/step - loss: 0.9184 - accuracy: 0.6801 - val_loss: 1.0608 - val_accuracy: 0.6248
Epoch 62/500
49/49 [==============================] - 2s 43ms/step - loss: 0.9129 - accuracy: 0.6783 - val_loss: 1.0181 - val_accuracy: 0.6444
Epoch 63/500
49/49 [==============================] - 2s 44ms/step - loss: 0.8924 - accuracy: 0.6860 - val_loss: 0.9633 - val_accuracy: 0.6617
Epoch 64/500
49/49 [==============================] - 2s 44ms/step - loss: 0.8952 - accuracy: 0.6853 - val_loss: 0.9727 - val_accuracy: 0.6577
Epoch 65/500

49/49 [==============================] - 2s 44ms/step - loss: 0.5354 - accuracy: 0.8138 - val_loss: 0.8947 - val_accuracy: 0.7077
Epoch 116/500
49/49 [==============================] - 2s 44ms/step - loss: 0.5591 - accuracy: 0.8060 - val_loss: 0.8655 - val_accuracy: 0.7149
Epoch 117/500
49/49 [==============================] - 2s 44ms/step - loss: 0.5394 - accuracy: 0.8126 - val_loss: 0.9095 - val_accuracy: 0.7057
Epoch 118/500
49/49 [==============================] - 2s 44ms/step - loss: 0.5325 - accuracy: 0.8141 - val_loss: 0.9027 - val_accuracy: 0.7081
Epoch 119/500
49/49 [==============================] - 2s 46ms/step - loss: 0.5219 - accuracy: 0.8156 - val_loss: 0.8881 - val_accuracy: 0.7187
Epoch 120/500
49/49 [==============================] - 2s 43ms/step - loss: 0.5194 - accuracy: 0.8182 - val_loss: 0.9234 - val_accuracy: 0.7018
Epoch 121/500
49/49 [==============================] - 2s 44ms/step - loss: 0.5178 - accuracy: 0.8189 - val_loss: 0.9216 - val_accuracy: 0.7052
Epoch 

Epoch 172/500
49/49 [==============================] - 2s 43ms/step - loss: 0.2823 - accuracy: 0.9006 - val_loss: 1.1539 - val_accuracy: 0.7039
Epoch 173/500
49/49 [==============================] - 2s 43ms/step - loss: 0.2702 - accuracy: 0.9053 - val_loss: 1.1968 - val_accuracy: 0.6977
Epoch 174/500
49/49 [==============================] - 2s 43ms/step - loss: 0.2678 - accuracy: 0.9053 - val_loss: 1.1740 - val_accuracy: 0.6980
Epoch 175/500
49/49 [==============================] - 2s 43ms/step - loss: 0.2627 - accuracy: 0.9089 - val_loss: 1.2504 - val_accuracy: 0.6934
Epoch 176/500
49/49 [==============================] - 2s 44ms/step - loss: 0.2794 - accuracy: 0.9023 - val_loss: 1.2123 - val_accuracy: 0.6947
Epoch 177/500
49/49 [==============================] - 2s 43ms/step - loss: 0.2507 - accuracy: 0.9121 - val_loss: 1.2505 - val_accuracy: 0.6936
Epoch 178/500
49/49 [==============================] - 2s 43ms/step - loss: 0.2608 - accuracy: 0.9067 - val_loss: 1.3131 - val_accuracy:

Epoch 229/500
49/49 [==============================] - 2s 43ms/step - loss: 0.0481 - accuracy: 0.9905 - val_loss: 1.6489 - val_accuracy: 0.7107
Epoch 230/500
49/49 [==============================] - 2s 44ms/step - loss: 0.0359 - accuracy: 0.9951 - val_loss: 1.7376 - val_accuracy: 0.7068
Epoch 231/500
49/49 [==============================] - 2s 43ms/step - loss: 0.0314 - accuracy: 0.9959 - val_loss: 1.8124 - val_accuracy: 0.7115
Epoch 232/500
49/49 [==============================] - 2s 44ms/step - loss: 0.0258 - accuracy: 0.9975 - val_loss: 1.8519 - val_accuracy: 0.7109
Epoch 233/500
49/49 [==============================] - 2s 43ms/step - loss: 0.0210 - accuracy: 0.9989 - val_loss: 1.9394 - val_accuracy: 0.7077
Epoch 234/500
49/49 [==============================] - 2s 44ms/step - loss: 0.0184 - accuracy: 0.9992 - val_loss: 1.9552 - val_accuracy: 0.7107
Epoch 235/500
49/49 [==============================] - 2s 43ms/step - loss: 0.0161 - accuracy: 0.9994 - val_loss: 2.0041 - val_accuracy:

Epoch 286/500
49/49 [==============================] - 2s 43ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 2.9232 - val_accuracy: 0.7104
Epoch 287/500
49/49 [==============================] - 2s 43ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 2.9316 - val_accuracy: 0.7094
Epoch 288/500
49/49 [==============================] - 2s 43ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 2.9396 - val_accuracy: 0.7104
Epoch 289/500
49/49 [==============================] - 2s 44ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 2.9510 - val_accuracy: 0.7093
Epoch 290/500
49/49 [==============================] - 2s 44ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 2.9551 - val_accuracy: 0.7102
Epoch 291/500
49/49 [==============================] - 2s 43ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 2.9642 - val_accuracy: 0.7101
Epoch 292/500
49/49 [==============================] - 2s 44ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 2.9732 - val_accuracy:

Epoch 342/500
49/49 [==============================] - 2s 44ms/step - loss: 6.3990e-04 - accuracy: 1.0000 - val_loss: 3.2843 - val_accuracy: 0.7095
Epoch 343/500
49/49 [==============================] - 2s 44ms/step - loss: 6.3303e-04 - accuracy: 1.0000 - val_loss: 3.2895 - val_accuracy: 0.7094
Epoch 344/500
49/49 [==============================] - 2s 43ms/step - loss: 6.2586e-04 - accuracy: 1.0000 - val_loss: 3.2931 - val_accuracy: 0.7099
Epoch 345/500
49/49 [==============================] - 2s 43ms/step - loss: 6.1919e-04 - accuracy: 1.0000 - val_loss: 3.2991 - val_accuracy: 0.7090
Epoch 346/500
49/49 [==============================] - 2s 43ms/step - loss: 6.1285e-04 - accuracy: 1.0000 - val_loss: 3.3078 - val_accuracy: 0.7092
Epoch 347/500
49/49 [==============================] - 2s 44ms/step - loss: 6.0662e-04 - accuracy: 1.0000 - val_loss: 3.3092 - val_accuracy: 0.7097
Epoch 348/500
49/49 [==============================] - 2s 45ms/step - loss: 6.0020e-04 - accuracy: 1.0000 - val_

49/49 [==============================] - 2s 43ms/step - loss: 3.9212e-04 - accuracy: 1.0000 - val_loss: 3.5029 - val_accuracy: 0.7084
Epoch 398/500
49/49 [==============================] - 2s 43ms/step - loss: 3.8932e-04 - accuracy: 1.0000 - val_loss: 3.5057 - val_accuracy: 0.7086
Epoch 399/500
49/49 [==============================] - 2s 43ms/step - loss: 3.8652e-04 - accuracy: 1.0000 - val_loss: 3.5090 - val_accuracy: 0.7088
Epoch 400/500
49/49 [==============================] - 2s 43ms/step - loss: 3.8342e-04 - accuracy: 1.0000 - val_loss: 3.5126 - val_accuracy: 0.7086
Epoch 401/500
49/49 [==============================] - 2s 44ms/step - loss: 3.8103e-04 - accuracy: 1.0000 - val_loss: 3.5151 - val_accuracy: 0.7082
Epoch 402/500
49/49 [==============================] - 2s 43ms/step - loss: 3.7800e-04 - accuracy: 1.0000 - val_loss: 3.5190 - val_accuracy: 0.7086
Epoch 403/500
49/49 [==============================] - 2s 43ms/step - loss: 3.7535e-04 - accuracy: 1.0000 - val_loss: 3.5204 -

49/49 [==============================] - 2s 43ms/step - loss: 2.7606e-04 - accuracy: 1.0000 - val_loss: 3.6606 - val_accuracy: 0.7078
Epoch 453/500
49/49 [==============================] - 2s 44ms/step - loss: 2.7447e-04 - accuracy: 1.0000 - val_loss: 3.6628 - val_accuracy: 0.7077
Epoch 454/500
49/49 [==============================] - 2s 44ms/step - loss: 2.7295e-04 - accuracy: 1.0000 - val_loss: 3.6646 - val_accuracy: 0.7071
Epoch 455/500
49/49 [==============================] - 2s 44ms/step - loss: 2.7154e-04 - accuracy: 1.0000 - val_loss: 3.6674 - val_accuracy: 0.7069
Epoch 456/500
49/49 [==============================] - 2s 43ms/step - loss: 2.7009e-04 - accuracy: 1.0000 - val_loss: 3.6705 - val_accuracy: 0.7071
Epoch 457/500
49/49 [==============================] - 2s 43ms/step - loss: 2.6861e-04 - accuracy: 1.0000 - val_loss: 3.6723 - val_accuracy: 0.7070
Epoch 458/500
49/49 [==============================] - 2s 43ms/step - loss: 2.6715e-04 - accuracy: 1.0000 - val_loss: 3.6773 -

In [ ]:
accuracies

In [ ]:

processed_accuracies = dict()
for key in accuracies.keys():   
    processed_accuracies[key] = accuracies[key][0].history
    
import pickle
file = open("dp_data/results/cifar10_accuracies_%.1f_%.1f_%.1f_%d"%(learning_rate, noise_multiplier, l2_norm_clip, batch_size),"wb")
pickle.dump(processed_accuracies, file)
file.close()

In [ ]:
assert(False)

In [ ]:
processed_accuracies

## Perturbed Label : skip


In [ ]:
def clip_noise(x, clip, noise_multiplier):
    def clip_features(x):
        return tf.clip_by_global_norm([x], clip)[0][0]
    clipped_features = tf.map_fn(clip_features, x)
    # Add noise to summed gradients.
    noise_stddev = clip * noise_multiplier
    noise = tf.random.normal(tf.shape(input=clipped_features), stddev=noise_stddev)
    return clipped_features + noise

noisy_train_data = tf.map_fn(lambda x: clip_noise(x, l2_norm_clip, noise_multiplier), train_data)

In [ ]:
accuracies

In [ ]:
processed_accuracies = dict()
for key in accuracies.keys():   
    processed_accuracies[key] = accuracies[key][0].history

In [ ]:
processed_accuracies

In [ ]:
import matplotlib.pyplot as plt
plt.plot(processed_accuracies['sphinx+mnist+untied_bias']['accuracy'])
plt.plot(processed_accuracies['sphinx+mnist+untied_bias']['val_accuracy'])
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
import pickle
file = open("dp_data/results/accuracies_%.1f_%.1f_%.1f_%d"%(learning_rate, noise_multiplier, l2_norm_clip, batch_size),"wb")
pickle.dump(processed_accuracies, file)
file.close()

In [ ]:
assert(False)

## get the step-epsilon curve

In [ ]:
import numpy as np
import pickle 

try:
    with open("dp_data/noise_epsilon_step") as f:
        epsilons = pickle.load(f)
except:
    training_data_size = 60000
    max_step = 25000
    epsilons = [[] for i in range(10)]
    for noise_multiplier in range(0, 10, 1):
        print("noise_multiplier:", noise_multiplier)
        for step in range(1, max_step, 10):
            epsilons[noise_multiplier].append(compute_epsilon(step, batch_size/training_data_size, noise_multiplier))
    file = open("dp_data/noise_epsilon_step_%d"%batch_size,"wb")
    pickle.dump(epsilons, file)
    file.close()

## Tied Bias

In [ ]:
# For tied bias
#for model_type in ["mnist", "mnist+tied_bias", "mnist+untied_bias"]:
for model_type in ["sphinx+mnist+tied_bias", "ALLnoise+mnist+tied_bias","mnist+tied_bias"]:
    file_name = model_type + "_accuracy"
    if model_type == "mnist+tied_bias+noise_input":
        dpsgd = False
    else:
        dpsgd = True
    model = build_models(model_type)
    if dpsgd:
        optimizer = FixedDPKerasSGDOptimizer(
            batch_size = batch_size,
            num_parameters = num_parameters,
            l2_norm_clip=l2_norm_clip,
            noise_multiplier=noise_multiplier,
            var_list = model.trainable_variables,
            #num_microbatches=batch_size//microbatch_size,
            microbatch_size = microbatch_size,
            learning_rate=learning_rate)
        # Compute vector of per-example loss rather than its mean over a minibatch.
        loss = tf.keras.losses.CategoricalCrossentropy(
            from_logits=True, reduction=tf.losses.Reduction.NONE)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
        loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # Compile model with Keras
    checkpoint_filepath = "./dp_data/"+file_name
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      save_weights_only=True,
      monitor='val_accuracy',
      mode='max',
      save_best_only=True)
    #early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    ##    monitor='val_loss', min_delta=0, patience=5, verbose=0,
    #    mode='auto', baseline=None, restore_best_weights=True
    #)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    # Train model with Keras
    if dpsgd:
        history = model.fit(train_data, train_labels,
                epochs=epochs,
                validation_data=(test_data, test_labels),
                batch_size=microbatch_size, 
                callbacks = [model_checkpoint_callback], workers=1) # , early_stopping_callback
    else:
        history = model.fit(train_data, train_labels,
                epochs=epochs,
                validation_data=(test_data, test_labels),
                batch_size=batch_size, 
                callbacks = [model_checkpoint_callback], workers=1) # , early_stopping_callback
    
    #evaluated_result = model.evaluate(
    #    x=test_data, y=test_labels, batch_size=None, verbose=1, sample_weight=None, steps=None,
    #    callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False,
    #    return_dict=True)
    if model_type not in accuracies.keys():
        accuracies[model_type] = []
    accuracies[model_type].append(history)
    #accuracies[model_type].append(evaluated_result["accuracy"])

# Compute the privacy budget expended.
#if dpsgd:
#    eps = compute_epsilon(epochs * 60000 // batch_size)
#    print('For delta=1e-5, the current epsilon is: %.2f' % eps)
#else:
#    print('Trained with vanilla non-private SGD optimizer')

In [ ]:
accuracies

In [ ]:
train_data

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

# Using set_dashes() to modify dashing of an existing line
for key in parameter_list.keys():
    line1, = ax.plot(parameter_list[key],accuracies[key],  label=key)
    line1.set_dashes([2, 2, 10, 2])  # 2pt line, 2pt break, 10pt line, 2pt break

ax.legend()
plt.show()

In [ ]:
ee
import pickle
result = open("result","wb")
pickle.dump(accuracies, result)
pickle.dump(parameter_list, result)
result.close()


In [ ]:
import pickle dd

In [ ]:
result = open("result", "rb")
accuracies = pickle.load(result)
parameter_list = pickle.load(result)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
len(num)

In [ ]:
parameter_list

In [ ]:

fig, ax = plt.subplots()

# Using set_dashes() to modify dashing of an existing line
line1, = ax.plot(parameter_list["linear"],accuracies["linear"],  label='Linear')
line1.set_dashes([2, 2, 10, 2])  # 2pt line, 2pt break, 10pt line, 2pt break

line2, = ax.plot(parameter_list["bias"],  accuracies["bias"], label='Bias')
line2.set_dashes([2, 2, 10, 2])  # 2pt line, 2pt break, 10pt line, 2pt break

ax.legend()
plt.show()

In [ ]:
#Test
a = [tf.constant([2., 3, 4.], shape=(3,), dtype=tf.float32), tf.constant([3., 4, 5], shape=(3,), dtype=tf.float32), tf.constant([2., 3, 4.], shape=(3,), dtype=tf.float32)]
b = tf.ones_like(a)
b

In [ ]:
tf.linalg.global_norm(a[:2])

In [ ]:
tf.clip_by_global_norm(a, 1)[0]

In [ ]:
tf.Variable(0)

In [ ]:
tf.